In [1]:
import numpy as np
import pandas as pd
import os
import shutil

In [8]:
def reorganise_gwhd(file_path,new_path):
    #csv dosyaları data frame ; images bilgileride dizi olarak alınıyor
    images_path=file_path+"\\images"
    test_path = file_path + "\\competition_test.csv"                          
    train_path = file_path + "\\competition_train.csv"                              
    val_path= file_path + "\\competition_val.csv"

    types_names=["train","test","val"]

    test_csv = pd.read_csv(test_path)
    train_csv = pd.read_csv(train_path)                  
    val_csv = pd.read_csv(val_path)
    images=os.listdir(images_path)

    #bütün etikteler karıştırılıyor bunun sebebi verinin bize sunulmuş halinde train test val yapılarında sınıf dağılımı eşit değil buda başarıyı önemli ölçüde olumsuz etkileyebilir

    all_csv=pd.concat([train_csv,test_csv,val_csv],ignore_index=True)
    shuffed_csv=all_csv.sample(frac=1).reset_index(drop=True)


    #karıştırılmış veri tekrar train,test,val olarak bölünüyor
    first_point=int(len(shuffed_csv)*0.6)
    second_point=int(first_point+((len(shuffed_csv)-first_point)/2))


    train=shuffed_csv[:first_point]
    test=shuffed_csv[first_point:second_point]
    val=shuffed_csv[second_point:]

    #buğday türleri types listesinde tutuluyor
    label_path=file_path+"\\"+"metadata_dataset.csv"
    labels=pd.read_csv(label_path)

    types=[]
    for label in labels.iterrows():
        type=label[1][0].split(';')[0]
        types.append(type.lower())

    new_images_path=new_path+"\\images"
    labels_path=new_path+"\\labels"
    os.makedirs(new_images_path)
    os.makedirs(labels_path)
    #her veri için görsel ve etiket gruplandırılması yapılıp yeni konumlarına yerleştiriliyor.
    all_datas=[train,test,val]
    for k in range(len(all_datas)):
        data=all_datas[k]
        for i in range(len(data)):
            type=data.iloc[i]["domain"]
            name=data.iloc[i]["image_name"]
            cordinates=data.iloc[i]["BoxesString"]

            image_file_path=new_images_path+"\\"+types_names[k]+"\\"+type
            if not os.path.exists(image_file_path):
                        os.makedirs(image_file_path)
            shutil.copy(images_path+"\\"+name,image_file_path)
            
            type_file_path=labels_path+"\\"+types_names[k]+"\\"+type
            if not os.path.exists(type_file_path):
                        os.makedirs(type_file_path)

            index=types.index(type.lower())
            new_text_path=type_file_path+"\\"+name.replace(".png", "")+".txt"
            with open(new_text_path,'w') as file:
                    for cordinate in cordinates.split(";"):
                        file.write(f"{str(index)} {cordinate}\n")

In [9]:
file_path="D:\\tarimda_yapay_zeka\\VERİLER\\gwhd_2021"  #dataset in mevcut konumu
new_path="D:\\tarimda_yapay_zeka\\VERİLER\\data"        #düzenlenmiş verilerin yeni konumu

reorganise_gwhd(file_path,new_path)

C:\Users\Mehmet\AppData\Local\Temp\ipykernel_18716\1434259899.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  type=label[1][0].split(';')[0]
